In [1]:
import os
import pandas as pd
import numpy as np
from datetime import timedelta


In [ ]:
loco_log=r"/Users/akshayarao/Downloads/RF_weak_-LS_mode_auto_tool_v1-main/17TH 39252"
#loco_id=39252

Data reading

In [ ]:
csvfolder=os.path.join(loco_log,"csv_files")

def read_csv_files(csvfolder,file_names):
    dataframes={}
    for file_name in file_names:
        matching_files = [f for f in os.listdir(csvfolder) if file_name in f and f.endswith(".csv")]
        if matching_files:
            file_path= os.path.join(csvfolder,matching_files[0])
            df=pd.read_csv(file_path)
            dataframes[file_name]=df
            print(f"successfully read {file_name}")
        else:
            print(f"failed to read {file_name}")
    return dataframes

file_names = ["LocoRadioRegStringLog", "LocoRadioStringLog", "StnRegLogV4", "StnRespStringLog"]    
csvfiles = read_csv_files(csvfolder, file_names) 
LocoRequest=csvfiles.get("LocoRadioStringLog")
StationResponse=csvfiles.get("StnRespStringLog")
LocoRegular= csvfiles.get("LocoRadioRegStringLog")
StationRegular= csvfiles.get("StnRegLogV4")
LocoRequest=LocoRequest[['FrameNo','TimeStamp','LocoId','Abs Location','Dir','Mode','Appr Stn','Tag-Id']]
StationResponse=StationResponse[['Type', 'Length', 'FrameNo', 'TimeStamp', 'StnId', 'Abs Lctn','Dest LocoId']]
LocoRegular=LocoRegular[['FrameNo', 'TimeStamp', 'LocoId','Abs Location','Dir', 'Emergency', 'Mode', 'Tag-Id','StPrfId']]
StationRegular = StationRegular[['FrameNo', 'TimeStamp', 'Stn-Id', 'DestLocoID', 'LastRefRFID', 'PktDir', 'Frame Offset','TrainSecType']]
loco_id = LocoRegular.loc[LocoRegular['Mode'] == "FS", 'LocoId'].iloc[0] # getting the loco-idwhere the mode is FS
def data_cleaning(df,loco_id):
    df=df.sort_values(by="FrameNo")
    df = df.drop_duplicates(subset=["FrameNo"])
    if 'LocoId' in df.columns:
        df=df[df["LocoId"]==loco_id] 
    elif 'DestLocoID' in df.columns:
        df=df[df["DestLocoID"]==loco_id]
    else:
        df=df[df["Dest LocoId"]==loco_id]
    df['TimeStamp'] = pd.to_datetime(df['TimeStamp'])
    return df    
LocoRequest=data_cleaning (LocoRequest,loco_id)
StationResponse=data_cleaning (StationResponse,loco_id)
LocoRegular=data_cleaning (LocoRegular,loco_id)
StationRegular =data_cleaning (StationRegular,loco_id)
print(loco_id)


successfully read LocoRadioRegStringLog
successfully read LocoRadioStringLog
successfully read StnRegLogV4
successfully read StnRespStringLog


/var/folders/rk/bqs7mvtn17s4_vhvqd9n39700000gn/T/ipykernel_54009/2687786312.py:36: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['TimeStamp'] = pd.to_datetime(df['TimeStamp'])
/var/folders/rk/bqs7mvtn17s4_vhvqd9n39700000gn/T/ipykernel_54009/2687786312.py:36: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['TimeStamp'] = pd.to_datetime(df['TimeStamp'])
/var/folders/rk/bqs7mvtn17s4_vhvqd9n39700000gn/T/ipykernel_54009/2687786312.py:36: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['TimeStamp'] = pd.to_datetime(df['TimeStamp'])
/var/folders/rk/bqs7mvtn17s4_vhvqd9n3970000

39252


In [4]:
print(StationRegular.columns)

Index(['FrameNo', 'TimeStamp', 'Stn-Id', 'DestLocoID', 'LastRefRFID', 'PktDir',
       'Frame Offset', 'TrainSecType'],
      dtype='object')


In [5]:
import pandas as pd
import numpy as np

class RFWeakASPBlank:
    def __init__(self, LocoRequest, StationResponse, LocoRegular, StationRegular, loco_id, direction):
        self.direction = direction
        self.loco_id = loco_id
        self.loco_rqst = LocoRequest[LocoRequest['Dir'] == direction]
        self.stn_resp = StationResponse
        self.loco_reg = LocoRegular[LocoRegular['Dir'] == direction]
        self.stn_reg = StationRegular[StationRegular['PktDir'] == direction]
        self.unique_stations = self.loco_rqst['Appr Stn'].unique().tolist()

    def after_registration_check(self, station, reg_df):
        print("Registration done.")

    def registration_check(self, station):
        try:
            Loco_req_MT= self.loco_rqst[self.loco_rqst['Appr Stn'] == station]
            lrqst_time = Loco_req_MT['TimeStamp'].iloc[0]
            
        except IndexError:
            print(f"No loco request found for station {station}")
            return
        modes = Loco_req_MT['Mode'].astype(str).str.upper()
        mt = Loco_req_MT[(modes == "LS") & (modes.shift(1) == "FS")]
        if not mt.empty :
            print(f"loco switched to ls mode in {station} at {mt['TimeStamp']} ")
        filtered_resp = self.stn_resp[
            (self.stn_resp['StnId'] == station) & 
            (self.stn_resp['TimeStamp'] >= lrqst_time)
        ]

        sresp_time = lreg_time = sreg_time = np.nan

        if not filtered_resp.empty:
            first_resp_time = filtered_resp['TimeStamp'].iloc[0]
            if first_resp_time - lrqst_time < pd.Timedelta(minutes=5):
                sresp_time = first_resp_time

                lreg_filter = self.loco_reg[
                    (self.loco_reg['TimeStamp'] >= sresp_time) & 
                    (self.loco_reg['StPrfId'] == 0)
                ]
                if not lreg_filter.empty:
                    lreg_time = lreg_filter['TimeStamp'].iloc[0]

                    sreg_filter = self.stn_reg[
                        (self.stn_reg['TimeStamp'] >= lreg_time) & 
                        (self.stn_reg['Stn-Id'] == station)
                    ]
                    if not sreg_filter.empty:
                        sreg_time = sreg_filter['TimeStamp'].iloc[0]

        process_steps = ['Loco request', 'Station response', 'Loco regular', 'Station regular']
        timestamps = [lrqst_time, sresp_time, lreg_time, sreg_time]
        reg_df = pd.DataFrame({'registration process': process_steps, 'time': timestamps})
        reg_df['time difference'] = reg_df['time'].diff().dt.total_seconds()
        #print(reg_df)

        if reg_df['time'].isnull().any() and not mt.empty:
            missing = reg_df[reg_df['time'].isnull()]['registration process'].tolist()
            print(f"=> Registration in {station} failed due to missing: {', '.join(missing)} and loco switched to LS mode")
        elif reg_df['time difference'].sum() <= 6:
            #print(f"=> Loco registration with {station} station was perfect")
            self.after_registration_check(station, reg_df)
        else:
            delays = reg_df[
                (reg_df['time difference'] > 2) & reg_df['time difference'].notnull()
            ]
            if not delays.empty:
                delay_msgs = [
                    f"{row['registration process']} delayed by {row['time difference']} secs at {row['time']}"
                    for _, row in delays.iterrows()
                ]
                #print(f"=> Registration delayed in {station} station due to {', and '.join(delay_msgs)}")
                self.after_registration_check(station, reg_df)

    def process(self):
        for station in self.unique_stations:
            print(f"\nChecking station: {station}")
            self.registration_check(station)
print("************ Nominal direction analysis ************")
rf_nom = RFWeakASPBlank(LocoRequest, StationResponse, LocoRegular, StationRegular, loco_id, direction="N")
rf_nom.process()

print("************ Reverse direction analysis ************")
rf_rev = RFWeakASPBlank(LocoRequest, StationResponse, LocoRegular, StationRegular, loco_id, direction="R")
rf_rev.process()

************ Nominal direction analysis ************

Checking station: 24089
Registration done.

Checking station: 33132
Registration done.

Checking station: 33131
Registration done.

Checking station: 33130
Registration done.

Checking station: 33129
Registration done.

Checking station: 33128
Registration done.

Checking station: 33127
Registration done.

Checking station: 33126
Registration done.

Checking station: 33125
Registration done.

Checking station: 33124
Registration done.

Checking station: 33123
Registration done.

Checking station: 33122
Registration done.

Checking station: 33121
Registration done.

Checking station: 33120
Registration done.

Checking station: 33119
Registration done.

Checking station: 33118
Registration done.

Checking station: 33117
Registration done.

Checking station: 33116
Registration done.

Checking station: 33115
Registration done.

Checking station: 33114
Registration done.

Checking station: 33113
Registration done.

Checking station: 3311

In [ ]:
import pandas as pd
import numpy as np

class RFWeakASPBlank:
    def __init__(self, LocoRequest, StationResponse, LocoRegular, StationRegular, loco_id, direction):
        self.direction = direction
        self.loco_id = loco_id
        self.loco_rqst = LocoRequest[LocoRequest['Dir'] == direction]
        self.stn_resp = StationResponse
        self.loco_reg = LocoRegular[LocoRegular['Dir'] == direction]
        self.stn_reg = StationRegular[StationRegular['PktDir'] == direction]
        self.unique_stations = self.loco_rqst['Appr Stn'].unique().tolist()

    def after_registration_check(self, station, reg_df,lreg_time,sreg_time):
        print("Registration done.")
        print(lreg_time)
        print(sreg_time)
        Aft_reg_sreg= self.stn_reg[self.stn_reg['Stn-Id']==station].copy()
        Aft_reg_sreg['TimeDifference']=Aft_reg_sreg['TimeStamp'].diff().dt.total_seconds()
        
        if (Aft_reg_sreg['TimeDifference'] <= 4).any():
            print(f"miss time: {Aft_reg_sreg['TimeDifference']}")
            print(f"previous time: {Aft_reg_sreg['TimeStamp'].iloc[-1]}")

    def registration_check(self, station):
        try:
            Loco_req_MT= self.loco_rqst[self.loco_rqst['Appr Stn'] == station]
            lrqst_time = Loco_req_MT['TimeStamp'].iloc[0]
            
        except IndexError:
            print(f"No loco request found for station {station}")
            return
        mt = Loco_req_MT[(Loco_req_MT['Mode'] == "LS") & (Loco_req_MT['Mode'].shift(1) == "FS")]
        
        
        if not mt.empty :
           print(f"loco switched to ls mode in {station} at {mt['TimeStamp']} ")
        filtered_resp = self.stn_resp[
            (self.stn_resp['StnId'] == station) & 
            (self.stn_resp['TimeStamp'] >= lrqst_time)
        ]

        sresp_time = lreg_time = sreg_time = np.nan

        if not filtered_resp.empty:
            first_resp_time = filtered_resp['TimeStamp'].iloc[0]
            if first_resp_time - lrqst_time < pd.Timedelta(minutes=5):
                sresp_time = first_resp_time

                lreg_filter = self.loco_reg[
                    (self.loco_reg['TimeStamp'] >= sresp_time) & 
                    (self.loco_reg['StPrfId'] == 0)
                ]
                if not lreg_filter.empty:
                    lreg_time = lreg_filter['TimeStamp'].iloc[0]

                    sreg_filter = self.stn_reg[
                        (self.stn_reg['TimeStamp'] >= lreg_time) & 
                        (self.stn_reg['Stn-Id'] == station)
                    ]
                    if not sreg_filter.empty:
                        sreg_time = sreg_filter['TimeStamp'].iloc[0]

        process_steps = ['Loco request', 'Station response', 'Loco regular', 'Station regular']
        timestamps = [lrqst_time, sresp_time, lreg_time, sreg_time]
        reg_df = pd.DataFrame({'registration process': process_steps, 'time': timestamps})
        reg_df['time difference'] = reg_df['time'].diff().dt.total_seconds()
        print(reg_df)

        if reg_df['time'].isnull().any() and not mt.empty:
            missing = reg_df[reg_df['time'].isnull()]['registration process'].tolist()
            print(f"=> Registration in {station} failed due to missing: {', '.join(missing)} and loco switched to LS mode at {mt['TimeStamp']}")
        elif reg_df['time difference'].sum() <= 6:
            #print(f"=> Loco registration with {station} station was perfect")
            self.after_registration_check(station, reg_df,lreg_time,sreg_time)
        else:
            delays = reg_df[
                (reg_df['time difference'] > 2) & reg_df['time difference'].notnull()
            ]
            if not delays.empty:
                delay_msgs = [
                    f"{row['registration process']} delayed by {row['time difference']} secs at {row['time']}"
                    for _, row in delays.iterrows()
                ]
                #print(f"=> Registration delayed in {station} station due to {', and '.join(delay_msgs)}")
                self.after_registration_check(station, reg_df,lreg_time,sreg_time)

    def process(self):
        for station in self.unique_stations:
            print(f"\nChecking station: {station}")
            self.registration_check(station)
print("************ Nominal direction analysis ************")
rf_nom = RFWeakASPBlank(LocoRequest, StationResponse, LocoRegular, StationRegular, loco_id, direction="N")
rf_nom.process()

print("************ Reverse direction analysis ****************************************************************************************")
rf_rev = RFWeakASPBlank(LocoRequest, StationResponse, LocoRegular, StationRegular, loco_id, direction="R")
rf_rev.process()

************ Nominal direction analysis ************

Checking station: 24089
  registration process                time  time difference
0         Loco request 2025-07-10 19:57:31              NaN
1     Station response 2025-07-10 19:57:33              2.0
2         Loco regular 2025-07-10 19:57:35              2.0
3      Station regular 2025-07-10 19:57:39              4.0
Registration done.
2025-07-10 19:57:35
2025-07-10 19:57:39
Miss time rows (<=4s gap):
                TimeStamp  TimeDifference
5015  2025-07-10 19:57:41             2.0
17855 2025-07-10 19:57:43             2.0
17856 2025-07-10 19:57:45             2.0
5017  2025-07-10 19:57:47             2.0
5018  2025-07-10 19:57:49             2.0
...                   ...             ...
18040 2025-07-10 20:02:55             2.0
18041 2025-07-10 20:02:57             2.0
18042 2025-07-10 20:02:59             2.0
5096  2025-07-10 20:03:01             2.0
18046 2025-07-10 20:03:05             4.0

[141 rows x 2 columns]
Previous